# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [57]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [58]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [59]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [60]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [61]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating Keyspace

In [62]:

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [63]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Table songs is created in Apache Cassandra DB, Primary Keys are SessionId and the partition key is itemInSession.


In [64]:
"""
try:
    session.execute("DROP TABLE Songs;")
    print("Table Dropped!!")
except Exception as e:
    print(e)
"""

query = "CREATE TABLE IF NOT EXISTS Songs"
query = query + "  (sessionId     INT, \
                    itemInSession INT ,    \
                    artist        VARCHAR, \
                    firstName     VARCHAR, \
                    gender        VARCHAR, \
                    lastName      VARCHAR, \
                    length        FLOAT,   \
                    level         VARCHAR, \
                    location      VARCHAR, \
                    song_title    VARCHAR, \
                    userId        INT,     \
                    PRIMARY KEY (sessionId,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Below code reads data from the event_datafile_new.csv into cassandra table Songs

In [65]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """ INSERT INTO Songs (sessionId, itemInSession, artist,firstName,gender,lastName,length,level,location,song_title,userId) VALUES """
        query = query + "(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),  int(line[3]), line[0], line[1], line[2],  line[4], float(line[5]), line[6], line[7], line[9], int(line[10]) ))

print("Data Load is Complete")

Data Load is Complete


#### Validate the data loaded into the Apache Cassandra table songs

In [66]:

query = "SELECT artist, length, song_title FROM  Songs WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.length, row.song_title)



Faithless 495.30731201171875 Music Matters (Mark Knight Dub)


#### Table songs holds the user_sessions related to user_sessions. Primary Key is userid, the partition key is sessionId amd CLUSTERING column is itemInSession.

In [67]:
"""
try:
    session.execute("DROP TABLE user_sessions;")
    print("Table Dropped!!")
except Exception as e:
    print(e)
"""

query = "CREATE TABLE IF NOT EXISTS user_sessions"
query = query + "  (userId        INT, \
                    sessionId     INT, \
                    itemInSession  INT, \
                    artist        VARCHAR, \
                    firstName     VARCHAR, \
                    gender        VARCHAR, \
                    lastName      VARCHAR, \
                    length        FLOAT, \
                    level         VARCHAR, \
                    location      VARCHAR, \
                    song_title    VARCHAR, \
                    Name          VARCHAR, \
                    PRIMARY KEY ((userId, sessionId), itemInSession));"

try:
    session.execute(query)
except Exception as e:
    print(e)

                    

#### Below code reads data from the event_datafile_new.csv into cassandra table user_sessions

In [68]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """ INSERT INTO user_sessions (userId, sessionId, itemInSession,artist,firstName,gender,lastName,length,level,location,song_title,Name) VALUES """
        query = query + "(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        lv_name=  line[1] + ' ' +line[4]
        session.execute(query, (int(line[10]), int(line[8]),  int(line[3]),  line[0], line[1], line[2], line[4], float(line[5]), line[6], line[7], line[9], lv_name ))

print("Data Load is Complete")

Data Load is Complete


#### Validate the data loaded into the Apache Cassandra table user_sessions

In [69]:
query = "SELECT artist, song_title, Name FROM  user_sessions WHERE userid = 10 and sessionid = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.name)
    #print(row)


Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Table songs holds the user_sessions related to song_playlist_session. Primary Key are song_title and userId

In [70]:
try:
    session.execute("DROP TABLE song_playlist_session;")
    print("Table Dropped!!")
except Exception as e:
    print(e)


query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + "  (song_title    VARCHAR, \
                    userId        INT, \
                    artist        VARCHAR, \
                    firstName     VARCHAR, \
                    gender        VARCHAR, \
                    itemInSession INT ,    \
                    lastName      VARCHAR, \
                    length        FLOAT, \
                    level         VARCHAR, \
                    location      VARCHAR, \
                    sessionId     INT, \
                    PRIMARY KEY (song_title,userId ))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

Error from server: code=2200 [Invalid query] message="unconfigured table song_playlist_session"


#### Below code reads data from the event_datafile_new.csv into cassandra table song_playlist_session

In [45]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """ INSERT INTO song_playlist_session (song_title,userId,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId) VALUES """
        query = query + "(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

        lv_name=  line[1] + ' ' +line[4]
      
        session.execute(query, (line[9], int(line[10]),line[0], line[1], line[2],  int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8])))

print("Data Load is Complete")

Data Load is Complete


#### Validate the data loaded into the Apache Cassandra table song_playlist_session

In [46]:
query = "SELECT firstname, lastname FROM  song_playlist_session WHERE song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [47]:
try:
    session.execute("DROP TABLE Songs;")
    session.execute("DROP TABLE user_sessions;")
    session.execute("DROP TABLE song_playlist_session;")
    print("Tables Dropped!!")
except Exception as e:
    print(e)

Tables Dropped!!


### Close the session and cluster connection¶

In [48]:
session.shutdown()
cluster.shutdown()